In [19]:
import datetime
import io
import sys
import unicodedata
from datetime import timezone

import numpy as np
import pandas as pd
import pybaseball
import requests
import scipy.stats as stats
from bs4 import BeautifulSoup
from pybaseball import statcast
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import LabelEncoder

pybaseball.cache.enable()

# Load in my own 40 man Roster Scraper
directory = '~/Desktop/Random-Projects/MLB/RosterScraper'
sys.path.append(directory)
from RosterScraper import RosterScraper

## Loads in StatCast ID so batter names show in the Statcast data and loads in a scraped DF with every 40 man roster

In [20]:
url = 'https://docs.google.com/spreadsheets/d/1JgczhD5VDQ1EiXqVG-blttZcVwbZd5_Ne_mefUGwJnk/pub?output=csv'
res = requests.get(url)
ID = pd.read_csv(io.BytesIO(res.content), sep=',')
ID.dropna(subset=['MLBID'], inplace=True)
ID['MLBID'] = ID['MLBID'].astype(int)

Rosters = RosterScraper()
BID = Rosters[Rosters["Position"] == "Batter"]
PID = Rosters[Rosters["Position"] == "Pitcher"]

## Creating functions for data manipulation so they can match when joining separate datasets

In [21]:
def convert_name(name):
    if name == 'Rockies':
        return 'COL'
    elif name == 'Reds':
        return 'CIN'
    elif name == 'Mariners':
        return 'SEA'
    elif name == 'Nationals':
        return 'WSH'
    elif name == 'Yankees':
        return 'NYY'
    elif name == 'Astros':
        return 'HOU'
    elif name == 'Red Sox':
        return 'BOS'
    elif name == 'Athletics':
        return 'OAK'
    elif name == 'Mets':
        return 'NYM'
    elif name == 'Braves':
        return 'ATL'
    elif name == 'Giants':
        return 'SF'
    elif name == 'Brewers':
        return 'MIL'
    elif name == 'Rays':
        return 'TB'
    elif name == 'Royals':
        return 'KC'
    elif name == 'White Sox':
        return 'CWS'
    elif name == 'Cubs':
        return 'CHC'
    elif name == 'Angels':
        return 'LAA'
    elif name == 'Tigers':
        return 'DET'
    elif name == 'Diamondbacks':
        return 'ARI'
    elif name == 'Guardians':
        return 'CLE'
    elif name == 'Orioles':
        return 'BAL'
    elif name == 'Twins':
        return 'MIN'
    elif name == 'Marlins':
        return 'MIA'
    elif name == 'Phillies':
        return 'PHI'
    elif name == 'Rangers':
        return 'TEX'
    elif name == 'Dodgers':
        return 'LAD'
    elif name == 'Padres':
        return 'SD'
    elif name == 'Pirates':
        return 'PIT'
    elif name == 'Blue Jays':
        return 'TOR'
    elif name == 'Cardinals':
        return 'STL'
    else:
        return np.nan
    
def flip_names(name):
    first_name, last_name = name.split(", ")
    return f"{last_name} {first_name}"

def replace_special_chars(text):
    return unicodedata.normalize('NFKD', text).encode('ascii', 'ignore').decode('utf-8')

def append_suffix_to_duplicates(df, column):
        seen = {}
        for idx, value in enumerate(df[column]):
            if value in seen:
                seen[value] += 1
                df.at[idx, column] = f"{value}2"
            else:
                seen[value] = 1

## Scraping the RotoGrinders website for daily pitchers and lineups

In [22]:
def getDKData2024():
    eastern_time = datetime.datetime.now(timezone.utc).astimezone(timezone(datetime.timedelta(hours=-5)))
    todaysdate = eastern_time.strftime("%m-%d-%Y")
    url = 'https://rotogrinders.com/lineups/mlb?site=draftkings'
    r = requests.get(url)
    soup = BeautifulSoup(r.text, 'lxml')

    gamelist = []
    gamecards = soup.findAll("div", {"class": "game-card-teams"})
    for x in gamecards:
        twoteams = x.findAll("span", {"class": "team-nameplate-mascot"})
        roadteam = convert_name(twoteams[0].text)
        hometeam = convert_name(twoteams[1].text)
        gamekey = "{}@{}".format(roadteam,hometeam)
        gamelist.append(gamekey)

    matchupsdf = pd.DataFrame()
    for game in gamelist:
        roadteam = game.split("@")[0]
        hometeam = game.split("@")[1]
        thisdf1 = pd.DataFrame({"Team": roadteam, "Opp": hometeam, "RoadTeam": roadteam, "HomeTeam": hometeam},index=[0])
        thisdf2 = pd.DataFrame({"Team": hometeam, "Opp": roadteam, "RoadTeam": roadteam, "HomeTeam": hometeam},index=[0])
        matchupsdf = pd.concat([matchupsdf,thisdf1,thisdf2])
        
    oppdict = dict(zip(matchupsdf.Team,matchupsdf.Opp))
    hometeamdict = dict(zip(matchupsdf.Team,matchupsdf.HomeTeam))
    roadteamdict = dict(zip(matchupsdf.Team,matchupsdf.RoadTeam))

    disabled_span_list = []
    for span in soup.findAll("span", {"class": "player-nameplate disabled"}):
        for a in span.findAll("a"):
            disabled_span_list.append(a.text)

    spdata = pd.DataFrame()
    for div in soup.findAll("span", {"class": "player-nameplate", "data-position": "SP"}):
        if "TBD" in str(div):
            playername = "TBD"
            pos = "SP"
            sal = 0
        else:
            for a in div.findAll('a', {'class': 'player-nameplate-name'}):
                playername = a.text.strip()

            strdiv = str(div)
            pos = strdiv[strdiv.find("data-position")+15:strdiv.find("data-salary")-2]
            sal = strdiv[strdiv.find("data-salary")+13:strdiv.find("<div class = 'player-nameplate-info'>")-3]
        try:
            ownership = strdiv[strdiv.find('<span class="small muted" data-auth="502">') + 42:strdiv.find('%')]
            ownership = ownership.replace("</span>", "")
            ownership = ownership.replace("</span", "")
            ownership = ownership.replace("</div>", "")
            ownership = ownership.replace(" ", "")
        except:
            ownership = np.nan

        thisspdata = pd.DataFrame([[playername, sal, ownership]], columns = ["Player", "Salary", "Ownership"])
        spdata = pd.concat([spdata, thisspdata])

    spdata['Player'] = spdata['Player'].replace('Luis Ortiz', 'Luis L. Ortiz')
    spdata['Player'] = spdata['Player'].replace('Mike King', 'Michael King')
    spdata['Player'] = spdata['Player'].replace('Robert Zastryzny', 'Rob Zastryzny')

    spdata2 = pd.merge(spdata, PID[["Name", "Team"]], left_on = ["Player"], right_on = ["Name"], how = "left").rename(columns = {"Team": "PitcherTeam"})
    spdata3 = pd.merge(spdata2, matchupsdf[["Team", "Opp"]], left_on = ["PitcherTeam"], right_on = ["Team"], how = "left").drop(columns = ["Team"])

    append_suffix_to_duplicates(spdata3, 'PitcherTeam')
    append_suffix_to_duplicates(spdata3, 'Opp')

    opp_spname_dict = dict(zip(spdata3.Opp, spdata3.Player))
    opp_spsal_dict = dict(zip(spdata3.Opp, spdata.Salary))
    opp_spown_dict = dict(zip(spdata3.Opp, spdata3.Ownership))

    ludf = pd.DataFrame()
    
    for li in soup.findAll("li", {"class": "lineup-card-player"}):
        for a in li.findAll("a", {"class": ["player-nameplate-name", "player-nameplate disabled"]}):
            playername = a.text

        listring = str(li)
        for span in li.find("span", {"class": "small"}):
            luspot = span.text
            luspot = luspot.replace("\n", "")
            luspot = luspot.strip()
            luspot = int(luspot)
        pos = listring[listring.find("data-position")+15:listring.find("data-salary")-2]
        sal = listring[listring.find("data-salary")+13:listring.find("<span class='small'>")-3]
        ownership = ownership.replace("</span>", "")
        ownership = ownership.replace("</span", "")
        ownership = ownership.replace("</li", "")
        ownership = ownership.replace("</div>", "")
        ownership = ownership.replace(" ", "")

        try:
            sal = int(sal)
        except:
            sal = 0
        thisludf = pd.DataFrame([[playername, luspot, sal, ownership]], columns = ["Player", "Spot", "Sal", "Ownership"])
        ludf = pd.concat([ludf, thisludf])

    ludf2 = pd.merge(ludf, BID[["Name", "Team"]], left_on = ["Player"], right_on = ["Name"], how = "left").rename(columns = {"Team": "BatterTeam"})
    ludf2['BatterTeam'] = ludf2['BatterTeam'].fillna(method='ffill')
    ludf2['HomeTeam'] = ludf2['BatterTeam'].map(hometeamdict)
    ludf2['RoadTeam'] = ludf2['BatterTeam'].map(roadteamdict)

    ludf2_teamlist = list(ludf2["BatterTeam"])

    dhteams = []
    for x in ludf2_teamlist:
        if ludf2_teamlist.count(x) > 11:
            if x in dhteams:
                pass
            else:
                dhteams.append(x)

    extract_dh = ludf2[ludf2["BatterTeam"].isin(dhteams)]
    new_ludf2 = ludf2[~ludf2["BatterTeam"].isin(dhteams)]

    new_team_list = []
    new_home_list = []
    new_road_list = []
    runcounter = 0

    for x, home, road in zip(extract_dh["BatterTeam"].astype(str), 
                         extract_dh["HomeTeam"].astype(str), 
                         extract_dh["RoadTeam"].astype(str)):
        if runcounter < 18:
            new_team_list.append(x)
            new_home_list.append(home)
            new_road_list.append(road)
            runcounter += 1
        else:
            new_team_list.append(x + "2")
            new_home_list.append(home + "2")
            new_road_list.append(road + "2")
            runcounter += 1

    extract_dh["BatterTeam"] = new_team_list
    extract_dh["HomeTeam"] = new_home_list
    extract_dh["RoadTeam"] = new_road_list

    ludf2 = pd.concat([extract_dh, new_ludf2])
    ludf2["Opp"] = ludf2["BatterTeam"].map(oppdict)
    ludf2['SP'] = ludf2['BatterTeam'].map(opp_spname_dict)
    ludf2['SPSal'] = ludf2['BatterTeam'].map(opp_spsal_dict)
    ludf2['SPOwnership'] = ludf2['BatterTeam'].map(opp_spown_dict)
    ludf2['Date'] = todaysdate
    ludf2['Time'] = np.nan

    ludf3 = ludf2[['BatterTeam','RoadTeam','HomeTeam','Time','Spot','Player','Sal','Ownership','Date', "SP"]]

    dkdata = ludf3.copy()

    try:
        checknan = dkdata[["BatterTeam", "SP"]]
        getnans = checknan[["SP"].isna()]
        if len(getnans) == 0:
            nonans = 1
            nanmapdict = {}
        else:
            nonans = 0
            getnans["SP"] = disabled_span_list
            nanmapdict = dict(zip(getnans.Team, getnans.SP))
    except:
        pass

    try:
        dkdata["SP"] = np.where(dkdata["SP"].isna(), dkdata["BatterTeam"].map(nanmapdict), dkdata["SP"])
    except:
        pass
    
    for i in range(1, len(dkdata) - 1):
        if dkdata.loc[i, 'BatterTeam'] != dkdata.loc[i-1, 'BatterTeam']:
            if dkdata.loc[i, 'BatterTeam'] != dkdata.loc[i+1, 'BatterTeam']:
                dkdata.loc[i, 'BatterTeam'] = np.nan
                dkdata.loc[i, 'HomeTeam'] = np.nan
                dkdata.loc[i, 'RoadTeam'] = np.nan
                dkdata.loc[i, 'SP'] = np.nan

    
    dkdata[["BatterTeam", "RoadTeam", "HomeTeam"]] = dkdata[["BatterTeam", "RoadTeam", "HomeTeam"]].fillna(method='ffill')
    dkdata = dkdata.drop_duplicates(subset = ["BatterTeam", "SP"], keep = "first")
    dkdata = dkdata.drop(columns = ["Time", "Sal", "Ownership"])

    dkdata['BatterTeam'] = dkdata['BatterTeam'].replace('ARI', 'AZ')
    dkdata['RoadTeam'] = dkdata['RoadTeam'].replace('ARI', 'AZ')
    dkdata['HomeTeam'] = dkdata['HomeTeam'].replace('ARI', 'AZ')

    dkdata['Date'] = pd.to_datetime(dkdata['Date'])
    dkdata['Date'] = dkdata['Date'].dt.strftime('%Y-%m-%d')
    dkdata = dkdata.set_index("Date")
    dkdata = dkdata[["BatterTeam", "RoadTeam", "HomeTeam", "SP"]]

    return(dkdata)

In [46]:
Train3 = pd.read_csv("~/Desktop/Random-Projects/MLB/Data/DailyPitcherModelTrainingData.csv")
Train3 = Train3.drop(Train3.columns[0], axis=1)

## Creating a DF where it only loads in the stats of starting pitchers

In [24]:
# Group by game and team identifiers
groupby_cols = ['game_date', 'BatterTeam', 'away_team', 'home_team']

# Function to keep only the starter's data
def keep_starter(group):
    starter_name = group['player_name'].iloc[0]
    return group[group['player_name'] == starter_name]

def count_outs(x):
    single_outs = ['other_out', 'strikeout', 'field_out', "force_out", 'fielders_choice', 'fielders_choice_out', "sac_fly", "sac_bunt", "caught_stealing_2b", "caught_stealing_3b", "caught_stealing_home", "pickoff_caught_stealing_2b",  "pickoff_caught_stealing_3b",  "pickoff_caught_stealing_home"]
    double_outs = ['double_play', 'strikeout_double_play', 'grounded_into_double_play', "sac_fly_double_play"]
    triple_outs = ['triple_play']
    
    outs = (x.isin(single_outs)).sum() + 2 * (x.isin(double_outs)).sum() + 3 * (x.isin(triple_outs)).sum()
    return outs

## Loads in today's data

In [25]:
TodaysData = getDKData2024()
TodaysData

/var/folders/bs/6lp27nzn3wvg1ygrkwrcl96w0000gn/T/ipykernel_88760/2850615655.py:101: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  ludf2['BatterTeam'] = ludf2['BatterTeam'].fillna(method='ffill')
/var/folders/bs/6lp27nzn3wvg1ygrkwrcl96w0000gn/T/ipykernel_88760/2850615655.py:137: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  extract_dh["BatterTeam"] = new_team_list
/var/folders/bs/6lp27nzn3wvg1ygrkwrcl96w0000gn/T/ipykernel_88760/2850615655.py:138: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pand

,BatterTeam,RoadTeam,HomeTeam,SP
Date,,,,
2024-08-10,TEX,TEX,NYY,Carlos Rodon
2024-08-10,NYY,TEX,NYY,Nathan Eovaldi
2024-08-10,TEX2,TEX2,NYY2,Gerrit Cole
2024-08-10,NYY2,TEX2,NYY2,Cody Bradford
2024-08-10,OAK,OAK,TOR,Yariel Rodriguez
2024-08-10,TOR,OAK,TOR,Osvaldo Bido
2024-08-10,DET,DET,SF,Logan Webb
2024-08-10,SF,DET,SF,Alex Faedo
2024-08-10,SD,SD,MIA,Roddery Munoz


In [26]:
eastern_time = datetime.datetime.now(timezone.utc).astimezone(timezone(datetime.timedelta(hours=-5)))
savant2024 = statcast(start_dt = "2024-03-28", end_dt = eastern_time.strftime("%Y-%m-%d"))
savant2024['game_date'] = pd.to_datetime(savant2024['game_date'])
savant2024['game_date'] = savant2024['game_date'].dt.strftime('%Y-%m-%d')
savant2024['BatterTeam'] = np.where(savant2024['inning_topbot'] == 'Top', savant2024['away_team'], savant2024['home_team'])
savant2024['PitcherTeam'] = np.where(savant2024['inning_topbot'] == 'Top', savant2024['home_team'], savant2024['away_team'])
savant2024 = pd.merge(savant2024, ID[["MLBID", "MLBNAME"]], left_on = 'batter', right_on = 'MLBID', how = 'left')
savant2024.dropna(subset=['MLBNAME'], inplace=True)
savant2024 = savant2024.drop_duplicates(subset = ["pitch_type", "game_date", "release_speed", "release_pos_x", "release_pos_z", "player_name"], keep='first')
savant2024["player_name"] = savant2024["player_name"].apply(flip_names)
savant2024['AwayRunsScored'] = savant2024['post_away_score'] - savant2024['away_score']
savant2024['HomeRunsScored'] = savant2024['post_home_score'] - savant2024['home_score']
savant2024 = savant2024[["game_date", "home_team", "away_team", "inning", "inning_topbot", "at_bat_number", "pitch_number", "pitch_type", "BatterTeam", "MLBNAME", "balls", "strikes", "outs_when_up", "events", "description", "bb_type", "hit_distance_sc", "launch_speed", "launch_angle", "estimated_ba_using_speedangle", "estimated_woba_using_speedangle", "woba_value", "p_throws", "PitcherTeam", "player_name", "delta_home_win_exp", "delta_run_exp", "away_score", "home_score", "AwayRunsScored", "HomeRunsScored"]].sort_values(by = ["game_date", "home_team", "away_team", "inning_topbot", "at_bat_number", "pitch_number"], ascending=[True, True, True, False, True, True])
savant2024 = savant2024.groupby(groupby_cols).apply(keep_starter).reset_index(drop = True)
savant2024["player_name"] = savant2024["player_name"].apply(replace_special_chars)

This is a large query, it may take a moment to complete


100%|██████████| 136/136 [01:20<00:00,  1.69it/s]
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/pybaseball/statcast.py:85: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  final_data = pd.concat(dataframe_list, axis=0).convert_dtypes(convert_string=False)
/var/folders/bs/6lp27nzn3wvg1ygrkwrcl96w0000gn/T/ipykernel_88760/1109087191.py:14: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  savant2024 = savant2024.groupby(groupby

## Does the same grouping as the training data at the various levels

In [27]:
Season1 = savant2024.groupby(["game_date", "BatterTeam", "away_team", "home_team", "inning", "at_bat_number", "MLBNAME", "player_name", "p_throws"]).agg(
    Pitches = ("pitch_number", "size"),
    PA = ('events', lambda x: (x.isin(['other_out', 'single', 'double', 'triple', 'home_run', 'strikeout', 'field_out', 'field_error', 'fielders_choice', 'double_play', 'fielders_choice_out', 'strikeout_double_play', 'triple_play', 'grounded_into_double_play'])).sum()),
    Outs = ('events', count_outs),
    Hit = ('events', lambda x: (x.isin(['single', 'double', 'triple', 'home_run'])).sum()),
    FB = ('bb_type', lambda x: (x == 'fly_ball').sum()),
    HR = ('events', lambda x: (x == 'home_run').sum()),
    HBP = ('events', lambda x: (x == 'hit_by_pitch').sum()),
    Balls = ('description', lambda x: (x.isin(["ball", "hit_by_pitch", "blocked_ball"])).sum()),
    BB = ('events', lambda x: (x == 'walk').sum()),
    CS = ('description', lambda x: (x == 'called_strike').sum()),
    Whiff = ('description', lambda x: (x.isin(["swinging_strike", "swinging_strike_blocked", "foul_tip"])).sum()),
    Strikes = ('description', lambda x: (x.isin(["called_strike", "swinging_strike", "foul", "swinging_strike_blocked", "foul_tip"])).sum()),
    K = ('events', lambda x: (x == 'strikeout').sum()),
    xBA = ("estimated_ba_using_speedangle", "mean"),
    xwOBA = ("estimated_woba_using_speedangle", "mean"),
    wOBA = ("woba_value", "mean"),
    RunExp = ("delta_run_exp", "mean"),
    AwayRunsScored = ("AwayRunsScored", "sum"),
    HomeRunsScored = ("HomeRunsScored", "sum")).reset_index().fillna(0)

In [28]:
Season2 = Season1.groupby(["game_date", "BatterTeam", "away_team", "home_team", "player_name", "p_throws"]).agg(
    Pitches = ("Pitches", "sum"),
    AB = ("at_bat_number", "size"),
    PA = ("PA", "sum"),
    Outs = ("Outs", "sum"),
    Hits = ("Hit", "sum"),
    FB = ("FB", "sum"),
    HR = ("HR", "sum"),
    HBP = ("HBP", "sum"),
    Balls = ("Balls", "sum"),
    BB = ("BB", "sum"),
    CS = ("CS", "sum"),
    Whiff = ("Whiff", "sum"),
    Strikes = ("Strikes", "sum"),
    K = ("K", "sum"),
    xBA = ("xBA", "mean"),
    xwOBA = ("xwOBA", "mean"),
    wOBA = ("wOBA", "mean"),
    RunExp = ("RunExp", "mean"),
    AwayRunsScored = ("AwayRunsScored", "sum"),
    HomeRunsScored = ("HomeRunsScored", "sum")).reset_index().fillna(0)

Season2["IP"] = Season2["Outs"] / 3
Season2['RA'] = np.where((Season2['HomeRunsScored'] > 0) & (Season2['BatterTeam'] == Season2['home_team']), Season2["HomeRunsScored"],
                        np.where((Season2['AwayRunsScored'] > 0) & (Season2['BatterTeam'] == Season2['away_team']),Season2["AwayRunsScored"], 0))
Season2["RA/9"] = (27 * Season2["RA"] / Season2["Outs"])
Season2 = Season2.drop(columns = ["AwayRunsScored", "HomeRunsScored"])

In [30]:
lgHR = 10048
lgFB = 58110

Season2['FIP'] = (13 * Season2['HR'] + 3 * (Season2['BB'] + Season2['HBP']) - 2 * Season2['K']) / (Season2['IP']) + 3.137
Season2['xFIP'] = (13 * (Season2['FB'] * (lgHR/lgFB * 0.58)) + 3 * (Season2['BB'] + Season2['HBP']) - 2 * Season2['K']) / (Season2['IP']) + 3.137

Season2['K%'] = round((Season2['K'] / Season2['AB']) * 100, 2)
Season2['BB%'] = round((Season2['BB'] / Season2['AB']) * 100, 2)
Season2['K-BB%'] = Season2["K%"] - Season2["BB%"]
Season2['Ball%'] = round((Season2['Balls'] / Season2['Pitches']) * 100, 2)
Season2['Strike%'] = round((Season2['Strikes'] / Season2['Pitches']) * 100, 2)
Season2['CS%'] = round((Season2['CS'] / Season2['Pitches']) * 100, 2)
Season2['Whiff%'] = round((Season2['Whiff'] / Season2['Pitches']) * 100, 2)
Season2["CSW"] = Season2["CS"] + Season2["Whiff"]
Season2['CSW%'] = round((Season2['CSW'] / Season2['Pitches']) * 100, 2)
Season2 = Season2.drop(columns=["CSW"])

In [31]:
window_size5 = 5
window_size10 = 10
window_size20 = 20

# Rolling 5 game pitch averages
Season2['Pitches5'] = Season2.groupby('player_name')['Pitches'].rolling(window=window_size5, min_periods=1).mean().reset_index(level=0, drop=True)
Season2 = Season2.drop(Season2[Season2['Pitches5'] < 40].index)
# Rolling 5 and 10 game outs averages
Season2['Outs5'] = Season2.groupby('player_name')['Outs'].rolling(window=window_size5, min_periods=1).mean().reset_index(level=0, drop=True)
Season2['Outs10'] = Season2.groupby('player_name')['Outs'].rolling(window=window_size10, min_periods=1).mean().reset_index(level=0, drop=True)
# Rolling 5 and 10 game expected batting averages
Season2['xBA5'] = Season2.groupby('player_name')['xBA'].rolling(window=window_size5, min_periods=1).mean().reset_index(level=0, drop=True)
Season2['xBA10'] = Season2.groupby('player_name')['xBA'].rolling(window=window_size10, min_periods=1).mean().reset_index(level=0, drop=True)
# Rolling 5 and 10 game expected wOBA averages
Season2['xwOBA5'] = Season2.groupby('player_name')['xwOBA'].rolling(window=window_size5, min_periods=1).mean().reset_index(level=0, drop=True)
Season2['xwOBA10'] = Season2.groupby('player_name')['xwOBA'].rolling(window=window_size10, min_periods=1).mean().reset_index(level=0, drop=True)
# Rolling 5 and 10 game wOBA averages
Season2['wOBA5'] = Season2.groupby('player_name')['wOBA'].rolling(window=window_size5, min_periods=1).mean().reset_index(level=0, drop=True)
Season2['wOBA10'] = Season2.groupby('player_name')['wOBA'].rolling(window=window_size10, min_periods=1).mean().reset_index(level=0, drop=True)
# Rolling 5 and 10 game RA averages
Season2['RA5'] = Season2.groupby('player_name')['RA/9'].rolling(window=window_size5, min_periods=1).mean().reset_index(level=0, drop=True)
Season2['RA10'] = Season2.groupby('player_name')['RA/9'].rolling(window=window_size10, min_periods=1).mean().reset_index(level=0, drop=True)
# Rolling 5 and 10 game FIP averages
Season2['FIP5'] = Season2.groupby('player_name')['FIP'].rolling(window=window_size5, min_periods=1).mean().reset_index(level=0, drop=True)
Season2['FIP10'] = Season2.groupby('player_name')['FIP'].rolling(window=window_size10, min_periods=1).mean().reset_index(level=0, drop=True)
# Rolling 5 and 10 game xFIP averages
Season2['xFIP5'] = Season2.groupby('player_name')['xFIP'].rolling(window=window_size5, min_periods=1).mean().reset_index(level=0, drop=True)
Season2['xFIP10'] = Season2.groupby('player_name')['xFIP'].rolling(window=window_size10, min_periods=1).mean().reset_index(level=0, drop=True)
# Rolling 5 and 10 game K% averages
Season2['K%5'] = Season2.groupby('player_name')['K%'].rolling(window=window_size5, min_periods=1).mean().reset_index(level=0, drop=True)
Season2['K%10'] = Season2.groupby('player_name')['K%'].rolling(window=window_size10, min_periods=1).mean().reset_index(level=0, drop=True)
# Rolling 5 and 10 game BB% averages
Season2['BB%5'] = Season2.groupby('player_name')['BB%'].rolling(window=window_size5, min_periods=1).mean().reset_index(level=0, drop=True)
Season2['BB%10'] = Season2.groupby('player_name')['BB%'].rolling(window=window_size10, min_periods=1).mean().reset_index(level=0, drop=True)
# Rolling 5 and 10 game K-BB% averages
Season2['K-BB%5'] = Season2.groupby('player_name')['K-BB%'].rolling(window=window_size5, min_periods=1).mean().reset_index(level=0, drop=True)
Season2['K-BB%10'] = Season2.groupby('player_name')['K-BB%'].rolling(window=window_size10, min_periods=1).mean().reset_index(level=0, drop=True)
# Rolling 5 game Ball% averages
Season2['Ball%5'] = Season2.groupby('player_name')['Ball%'].rolling(window=window_size5, min_periods=1).mean().reset_index(level=0, drop=True)
# Rolling 5 game Strike% averages
Season2['Strike%5'] = Season2.groupby('player_name')['Strike%'].rolling(window=window_size5, min_periods=1).mean().reset_index(level=0, drop=True)
# Rolling 5 and 10 game Called Strike% averages
Season2['CS%5'] = Season2.groupby('player_name')['CS%'].rolling(window=window_size5, min_periods=1).mean().reset_index(level=0, drop=True)
Season2['CS%10'] = Season2.groupby('player_name')['CS%'].rolling(window=window_size10, min_periods=1).mean().reset_index(level=0, drop=True)
# Rolling 5 and 10 game Whiff% averages
Season2['Whiff%5'] = Season2.groupby('player_name')['Whiff%'].rolling(window=window_size5, min_periods=1).mean().reset_index(level=0, drop=True)
Season2['Whiff%10'] = Season2.groupby('player_name')['Whiff%'].rolling(window=window_size10, min_periods=1).mean().reset_index(level=0, drop=True)
# Rolling 5 and 10 game Called Strike plus Whiff% averages
Season2['CSW%5'] = Season2.groupby('player_name')['CSW%'].rolling(window=window_size5, min_periods=1).mean().reset_index(level=0, drop=True)
Season2['CSW%10'] = Season2.groupby('player_name')['CSW%'].rolling(window=window_size10, min_periods=1).mean().reset_index(level=0, drop=True)

Season3 = Season2.drop(columns = ["FB", "Balls", "HBP", "CS", "Whiff", "Strikes", 'Ball%', 'Strike%', 'CS%', 'Whiff%', 'CSW%', "RA/9"])
Season3 = Season3.rename(columns={'away_team': 'RoadTeam', 'home_team': 'HomeTeam', "player_name": "SP"})

## Grouping everything to get season and rolling averages for pitchers

In [32]:
Season4 = Season3.groupby(["SP", "p_throws"]).agg(
    Starts = ("IP", "size"),
    Pitches = ("Pitches", "mean"),
    AB = ("AB", "mean"),
    PA = ("PA", "mean"),
    Outs = ("Outs", "mean"),
    Hits = ("Hits", "mean"),
    HR = ("HR", "mean"),
    BB = ("BB", "mean"),
    K = ("K", "mean"),
    RA = ("RA", "mean"),
    xBA = ("xBA", "mean"),
    xwOBA = ("xwOBA", "mean"),
    wOBA = ("wOBA", "mean"),
    RunExp = ("RunExp", "mean"),
    FIP = ("FIP", "mean"),
    xFIP = ("xFIP", "mean"),
    Kpercent = ("K%", "mean"),
    BBpercent = ("BB%", "mean"),
    KminusBBpercent = ("K-BB%", "mean"),
    Pitches5 =  ("Pitches5", "last"),
    Outs5 =  ("Outs5", "last"),
    Outs10 = ("Outs10", "last"),
    xBA5 =  ("xBA5", "last"),
    xBA10 = ("xBA10", "last"),
    xwOBA5 =  ("xwOBA5", "last"),
    xwOBA10 = ("xwOBA10", "last"),
    wOBA5 =  ("wOBA5", "last"),
    wOBA10 = ("wOBA10", "last"),
    RA5 = ("RA5", "last"),
    RA10 = ("RA10", "last"),
    FIP5 = ("FIP5", "last"),
    FIP10 = ("FIP10", "last"),
    xFIP5 = ("xFIP5", "last"),
    xFIP10 = ("xFIP10", "last"),
    Kpercent5 = ("K%5", "last"),
    Kpercent10 = ("K%10", "last"),
    BBpercent5 = ("BB%5", "last"),
    BBpercent10 = ("BB%10", "last"),
    KminusBBpercent5 = ("K-BB%5", "last"),
    KminusBBpercent10 = ("K-BB%10", "last"),
    Ballpercent5 = ("Ball%5", "last"),
    Strikepercent5 = ("Strike%5", "last"),
    CSpercent5 = ("CS%5", "last"),
    CSpercent10 = ("CS%10", "last"),
    Whiffpercent5 = ("Whiff%5", "last"),
    Whiffpercent10 = ("Whiff%10", "last"),
    CSWpercent5 = ("CSW%5", "last"),
    CSWpercent10 = ("CSW%10", "last")).reset_index().fillna(0)

Season4.rename(columns={col: col.replace('percent', '%') for col in Season4.columns if 'percent' in col}, inplace=True)
Season4.rename(columns={col: col.replace('minus', '-') for col in Season4.columns if 'minus' in col}, inplace=True)

In [33]:
BatterSeason1 = savant2024.groupby(["game_date", "BatterTeam", "away_team", "home_team", "inning", "at_bat_number", "MLBNAME", "player_name", "p_throws"]).agg(
    Pitches = ("pitch_number", "size"),
    PA = ('events', lambda x: (x.isin(['other_out', 'single', 'double', 'triple', 'home_run', 'strikeout', 'field_out', 'field_error', 'fielders_choice', 'double_play', 'fielders_choice_out', 'strikeout_double_play', 'triple_play', 'grounded_into_double_play'])).sum()),
    Outs = ('events', count_outs),
    FB = ('bb_type', lambda x: (x == 'fly_ball').sum()),
    HR = ('events', lambda x: (x == 'home_run').sum()),
    HBP = ('events', lambda x: (x == 'hit_by_pitch').sum()),
    Balls = ('description', lambda x: (x.isin(["ball", "hit_by_pitch", "blocked_ball"])).sum()),
    BB = ('events', lambda x: (x == 'walk').sum()),
    CS = ('description', lambda x: (x == 'called_strike').sum()),
    Whiff = ('description', lambda x: (x.isin(["swinging_strike", "swinging_strike_blocked", "foul_tip"])).sum()),
    Strikes = ('description', lambda x: (x.isin(["called_strike", "swinging_strike", "foul", "swinging_strike_blocked", "foul_tip"])).sum()),
    K = ('events', lambda x: (x == 'strikeout').sum()),
    xBA = ("estimated_ba_using_speedangle", "mean"),
    xwOBA = ("estimated_woba_using_speedangle", "mean"),
    wOBA = ("woba_value", "mean"),
    RunExp = ("delta_run_exp", "mean"),
    AwayRunsScored = ("AwayRunsScored", "sum"),
    HomeRunsScored = ("HomeRunsScored", "sum")).reset_index().fillna(0)

In [34]:
BatterSeason2 = BatterSeason1.groupby(["game_date", "BatterTeam", "away_team", "home_team", "p_throws"]).agg(
    Pitches=("Pitches", "sum"),
    AB=("at_bat_number", "size"),
    PA=("PA", "sum"),
    Outs=("Outs", "sum"),
    FB=("FB", "sum"),
    HR=("HR", "sum"),
    HBP=("HBP", "sum"),
    Balls=("Balls", "sum"),
    BB=("BB", "sum"),
    CS=("CS", "sum"),
    Whiff=("Whiff", "sum"),
    Strikes=("Strikes", "sum"),
    K=("K", "sum"),
    xBA=("xBA", "mean"),
    xwOBA=("xwOBA", "mean"),
    wOBA=("wOBA", "mean"),
    RunExp=("RunExp", "mean"),
    AwayRunsScored=("AwayRunsScored", "sum"),
    HomeRunsScored=("HomeRunsScored", "sum")).reset_index().fillna(0)

BatterSeason2['RA'] = np.where((BatterSeason2['HomeRunsScored'] > 0) & (BatterSeason2['BatterTeam'] == BatterSeason2['home_team']),
    BatterSeason2["HomeRunsScored"], np.where((BatterSeason2['AwayRunsScored'] > 0) & (BatterSeason2['BatterTeam'] == BatterSeason2['away_team']), BatterSeason2["AwayRunsScored"], 0))
BatterSeason2["R/9"] = (27 * BatterSeason2["RA"] / BatterSeason2["Outs"])
BatterSeason2 = BatterSeason2.drop(columns=["AwayRunsScored", "HomeRunsScored", "RA"])

In [35]:
BatterSeason2['FIP'] = (13 * BatterSeason2['HR'] + 3 * (BatterSeason2['BB'] + BatterSeason2['HBP']) - 2 * BatterSeason2['K']) / (BatterSeason2['Outs'] / 3) + 3.137
BatterSeason2['xFIP'] = (13 * (BatterSeason2['FB'] * (lgHR/lgFB * 0.58)) + 3 * (BatterSeason2['BB'] + BatterSeason2['HBP']) - 2 * BatterSeason2['K']) / (BatterSeason2['Outs'] / 3) + 3.137

BatterSeason2['K%'] = round((BatterSeason2['K'] / BatterSeason2['AB']) * 100, 2)
BatterSeason2['BB%'] = round((BatterSeason2['BB'] / BatterSeason2['AB']) * 100, 2)
BatterSeason2['K-BB%'] = BatterSeason2["K%"] - BatterSeason2["BB%"]
BatterSeason2['Ball%'] = round((BatterSeason2['Balls'] / BatterSeason2['Pitches']) * 100, 2)
BatterSeason2['Strike%'] = round((BatterSeason2['Strikes'] / BatterSeason2['Pitches']) * 100, 2)
BatterSeason2['CS%'] = round((BatterSeason2['CS'] / BatterSeason2['Pitches']) * 100, 2)
BatterSeason2['Whiff%'] = round((BatterSeason2['Whiff'] / BatterSeason2['Pitches']) * 100, 2)
BatterSeason2["CSW"] = BatterSeason2["CS"] + BatterSeason2["Whiff"]
BatterSeason2['CSW%'] = round((BatterSeason2['CSW'] / BatterSeason2['Pitches']) * 100, 2)
BatterSeason2 = BatterSeason2.drop(columns = ["CSW"])

In [36]:
# Rolling 5 and 10 game expected batting averages
Season4['bxBA5'] = BatterSeason2.groupby(['BatterTeam', "p_throws"])['xBA'].rolling(window=window_size5, min_periods=1).mean().reset_index(level=[0,1], drop=True)
Season4['bxBA10'] = BatterSeason2.groupby(['BatterTeam', "p_throws"])['xBA'].rolling(window=window_size10, min_periods=1).mean().reset_index(level=[0,1], drop=True)
# Rolling 5 and 10 game expected wOBA averages
Season4['bxwOBA5'] = BatterSeason2.groupby(['BatterTeam', "p_throws"])['xwOBA'].rolling(window=window_size5, min_periods=1).mean().reset_index(level=[0,1], drop=True)
Season4['bxwOBA10'] = BatterSeason2.groupby(['BatterTeam', "p_throws"])['xwOBA'].rolling(window=window_size10, min_periods=1).mean().reset_index(level=[0,1], drop=True)
# Rolling 5 and 10 game wOBA averages
Season4['bwOBA5'] = BatterSeason2.groupby(['BatterTeam', "p_throws"])['wOBA'].rolling(window=window_size5, min_periods=1).mean().reset_index(level=[0,1], drop=True)
Season4['bwOBA10'] = BatterSeason2.groupby(['BatterTeam', "p_throws"])['wOBA'].rolling(window=window_size10, min_periods=1).mean().reset_index(level=[0,1], drop=True)
# Rolling 5 and 10 game RA averages
Season4['bRS5'] = BatterSeason2.groupby(['BatterTeam', "p_throws"])['R/9'].rolling(window=window_size5, min_periods=1).mean().reset_index(level=[0,1], drop=True)
Season4['bRS10'] = BatterSeason2.groupby(['BatterTeam', "p_throws"])['R/9'].rolling(window=window_size10, min_periods=1).mean().reset_index(level=[0,1], drop=True)
# Rolling 5 and 10 game FIP averages
Season4['bFIP5'] = BatterSeason2.groupby(['BatterTeam', "p_throws"])['FIP'].rolling(window=window_size5, min_periods=1).mean().reset_index(level=[0,1], drop=True)
Season4['bFIP10'] = BatterSeason2.groupby(['BatterTeam', "p_throws"])['FIP'].rolling(window=window_size10, min_periods=1).mean().reset_index(level=[0,1], drop=True)
# Rolling 5 and 10 game xFIP averages
Season4['bxFIP5'] = BatterSeason2.groupby(['BatterTeam', "p_throws"])['xFIP'].rolling(window=window_size5, min_periods=1).mean().reset_index(level=[0,1], drop=True)
Season4['bxFIP10'] = BatterSeason2.groupby(['BatterTeam', "p_throws"])['xFIP'].rolling(window=window_size10, min_periods=1).mean().reset_index(level=[0,1], drop=True)
# Rolling 5 and 10 game K% averages
Season4['bK%5'] = BatterSeason2.groupby(['BatterTeam', "p_throws"])['K%'].rolling(window=window_size5, min_periods=1).mean().reset_index(level=[0,1], drop=True)
Season4['bK%10'] = BatterSeason2.groupby(['BatterTeam', "p_throws"])['K%'].rolling(window=window_size10, min_periods=1).mean().reset_index(level=[0,1], drop=True)
# Rolling 5 and 10 game BB% averages
Season4['bBB%5'] = BatterSeason2.groupby(['BatterTeam', "p_throws"])['BB%'].rolling(window=window_size5, min_periods=1).mean().reset_index(level=[0,1], drop=True)
Season4['bBB%10'] = BatterSeason2.groupby(['BatterTeam', "p_throws"])['BB%'].rolling(window=window_size10, min_periods=1).mean().reset_index(level=[0,1], drop=True)
# Rolling 5 and 10 game K-BB% averages
Season4['bK-BB%5'] = BatterSeason2.groupby(['BatterTeam', "p_throws"])['K-BB%'].rolling(window=window_size5, min_periods=1).mean().reset_index(level=[0,1], drop=True)
Season4['bK-BB%10'] = BatterSeason2.groupby(['BatterTeam', "p_throws"])['K-BB%'].rolling(window=window_size10, min_periods=1).mean().reset_index(level=[0,1], drop=True)
# Rolling 5 game Ball% averages
Season4['bBall%5'] = BatterSeason2.groupby(['BatterTeam', "p_throws"])['Ball%'].rolling(window=window_size5, min_periods=1).mean().reset_index(level=[0,1], drop=True)
# Rolling 5 game Strike% averages
Season4['bStrike%5'] = BatterSeason2.groupby(['BatterTeam', "p_throws"])['Strike%'].rolling(window=window_size5, min_periods=1).mean().reset_index(level=[0,1], drop=True)
# Rolling 5 and 10 game Called Strike% averages
Season4['bCS%5'] = BatterSeason2.groupby(['BatterTeam', "p_throws"])['CS%'].rolling(window=window_size5, min_periods=1).mean().reset_index(level=[0,1], drop=True)
Season4['bCS%10'] = BatterSeason2.groupby(['BatterTeam', "p_throws"])['CS%'].rolling(window=window_size10, min_periods=1).mean().reset_index(level=[0,1], drop=True)
# Rolling 5 and 10 game Whiff% averages
Season4['bWhiff%5'] = BatterSeason2.groupby(['BatterTeam', "p_throws"])['Whiff%'].rolling(window=window_size5, min_periods=1).mean().reset_index(level=[0,1], drop=True)
Season4['bWhiff%10'] = BatterSeason2.groupby(['BatterTeam', "p_throws"])['Whiff%'].rolling(window=window_size10, min_periods=1).mean().reset_index(level=[0,1], drop=True)
# Rolling 5 and 10 game Called Strike plus Whiff% averages
Season4['bCSW%5'] = BatterSeason2.groupby(['BatterTeam', "p_throws"])['CSW%'].rolling(window=window_size5, min_periods=1).mean().reset_index(level=[0,1], drop=True)
Season4['bCSW%10'] = BatterSeason2.groupby(['BatterTeam', "p_throws"])['CSW%'].rolling(window=window_size10, min_periods=1).mean().reset_index(level=[0,1], drop=True)

## Joining both the pitcher averages and the batter rolling averages

In [47]:
TodaysData.dropna(subset=['SP'], inplace=True)
TodaysData1 = pd.merge(TodaysData, Season4[['SP', 'p_throws', 'Starts', 'Pitches', 'AB', 'PA', 'Outs', "Hits", 'HR', 'BB',
       'K', 'RA', 'xBA', 'xwOBA', 'wOBA', 'RunExp', 'FIP', 'xFIP', 'K%', 'BB%',
       'K-BB%', 'Pitches5', 'Outs5', 'Outs10', 'xBA5', 'xBA10', 'xwOBA5',
       'xwOBA10', 'wOBA5', 'wOBA10', 'RA5', 'RA10', 'FIP5', 'FIP10', 'xFIP5',
       'xFIP10', 'K%5', 'K%10', 'BB%5', 'BB%10', 'K-BB%5', 'K-BB%10', 'Ball%5',
       'Strike%5', 'CS%5', 'CS%10', 'Whiff%5', 'Whiff%10', 'CSW%5', 'CSW%10',
       'bxBA5', 'bxBA10', 'bxwOBA5', 'bxwOBA10', 'bwOBA5', 'bwOBA10', 'bRS5',
       'bRS10', 'bFIP5', 'bFIP10', 'bxFIP5', 'bxFIP10', 'bK%5', 'bK%10',
       'bBB%5', 'bBB%10', 'bK-BB%5', 'bK-BB%10', 'bBall%5', 'bStrike%5',
       'bCS%5', 'bCS%10', 'bWhiff%5', 'bWhiff%10', 'bCSW%5', 'bCSW%10']], left_on = ['SP'], right_on = ['SP'], how = 'left')

# If no 2024 savant data exists then gives them the league averages from 2022-23
TrainMeans = Train3.drop(['BatterTeam', 'RoadTeam', "HomeTeam", "SP", "p_throws"], axis=1).mean()
TodaysData1 = TodaysData1.fillna(TrainMeans)
TodaysData1 = TodaysData1.replace([float('inf'), -float('inf')], 5)

## Encodes the teams and players allowing to be fed into the algorithms

In [54]:
# Ensure Train5 and TodaysData2 are copies of Train4 and TodaysData1 respectively
Train5 = Train3.copy()
TodaysData2 = TodaysData1.copy()

# Dictionary to store the label encoders
label_encoders = {}

# Encode non-numeric columns in Train4
non_numeric_columns_train = Train5.select_dtypes(exclude=['float64', 'int64']).columns
for col in non_numeric_columns_train:
    label_encoder = LabelEncoder()
    Train5[col] = label_encoder.fit_transform(Train5[col])
    label_encoders[col] = label_encoder

# Ensure all non-numeric columns in Train4 are in TodaysData1
for col in non_numeric_columns_train:
    if col not in TodaysData2.columns:
        print(f"Warning: Column {col} from training data is not present in today's data.")
        # Adding the missing column with a default value
        TodaysData2[col] = 536

# Encode non-numeric columns in TodaysData1 using the same encoders
non_numeric_columns_today = TodaysData2.select_dtypes(exclude=['float64', 'int64']).columns
for col in non_numeric_columns_today:
    if col in label_encoders:
        label_encoder = label_encoders[col]
        unique_values = set(label_encoder.classes_)
        encoded_values = []
        for item in TodaysData2[col]:
            if item in unique_values:
                encoded_values.append(label_encoder.transform([item])[0])
            else:
                encoded_values.append(536)  # Using 536 as a placeholder for unknown categories
        TodaysData2[col] = encoded_values
    else:
        print(f"Warning: Column {col} is not present in the training data.")
        # Fit a new label encoder for columns not present in Train4, but be cautious with this
        label_encoder = LabelEncoder()
        TodaysData2[col] = label_encoder.fit_transform(TodaysData2[col])
        label_encoders[col] = label_encoder

## K model

In [55]:
TrainFeatures = Train5.drop(columns = ["K"]).values.reshape(-1, 77)
TrainLabel = Train5["K"].values.ravel()
TodayFeatures = TodaysData2.drop(columns = ["Starts", "K"]).values.reshape(-1, 77)

rf_regressor = RandomForestRegressor(n_estimators = 152, max_depth = 15, min_samples_leaf = 4)
rf_regressor.fit(TrainFeatures, TrainLabel)
RFpred = rf_regressor.predict(TodayFeatures)

TodaysData2["xK"] = RFpred

## BB Model

In [56]:
TrainFeatures = Train5.drop(columns = ["BB"]).values.reshape(-1, 77)
TrainLabel = Train5["BB"].values.ravel()
TodayFeatures = TodaysData2.drop(columns = ["Starts", "BB", "xK"]).values.reshape(-1, 77)

rf_regressor = RandomForestRegressor(n_estimators = 152, max_depth = 15, min_samples_leaf = 3)
rf_regressor.fit(TrainFeatures, TrainLabel)
RFpred = rf_regressor.predict(TodayFeatures)

TodaysData2["xBB"] = RFpred

## Hits Model

In [57]:
TrainFeatures = Train5.drop(columns = ["Hits"]).values.reshape(-1, 77)
TrainLabel = Train5["Hits"].values.ravel()
TodayFeatures = TodaysData2.drop(columns = ["Starts", "BB", "xK", "xBB"]).values.reshape(-1, 77)

rf_regressor = RandomForestRegressor(n_estimators = 152, max_depth = 15, min_samples_leaf = 3)
rf_regressor.fit(TrainFeatures, TrainLabel)
RFpred = rf_regressor.predict(TodayFeatures)

TodaysData2["xHA"] = RFpred

## RA Model

In [58]:
TrainFeatures = Train5.drop(columns = ["RA"]).values.reshape(-1, 77)
TrainLabel = Train5["RA"].values.ravel()
TodayFeatures = TodaysData2.drop(columns = ["Starts", "BB", "xK", "xBB", "xHA"]).values.reshape(-1, 77)

rf_regressor = RandomForestRegressor(n_estimators = 152, max_depth = 15, min_samples_leaf = 3)
rf_regressor.fit(TrainFeatures, TrainLabel)
RFpred = rf_regressor.predict(TodayFeatures)

TodaysData2["xRA"] = RFpred

## IP model

In [59]:
TrainFeatures = Train5.drop(columns = ["Outs"]).values.reshape(-1, 77)
TrainLabel = Train5["Outs"].values.ravel()
TodayFeatures = TodaysData2.drop(columns = ["Starts", "Outs", "xK", "xBB", "xHA", "xRA"]).values.reshape(-1, 77)

rf_regressor = RandomForestRegressor(n_estimators = 152, max_depth = 15, min_samples_leaf = 3)
rf_regressor.fit(TrainFeatures, TrainLabel)
RFpred = rf_regressor.predict(TodayFeatures)

TodaysData2["xOuts"] = RFpred

## Reverse encodes today's data so it can be understood

In [60]:
for col in non_numeric_columns_today:
    if col in label_encoders:
        label_encoder = label_encoders[col]
        # Handling default value of 536
        TodaysData2[col] = TodaysData2[col].apply(lambda x: label_encoder.inverse_transform([x])[0] if x != 536 else np.nan)

TodaysData2["SP"].fillna(TodaysData1["SP"], inplace = True)

/var/folders/bs/6lp27nzn3wvg1ygrkwrcl96w0000gn/T/ipykernel_88760/2051378488.py:7: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  TodaysData2["SP"].fillna(TodaysData1["SP"], inplace = True)


In [61]:
def add_2_to_duplicates(df):
    mask = df.duplicated(subset=['BatterTeam', 'RoadTeam', 'HomeTeam'], keep='first')
    
    df.loc[mask, 'BatterTeam'] += '2'
    df.loc[mask, 'RoadTeam'] += '2'
    df.loc[mask, 'HomeTeam'] += '2'
    
    return df

TodaysData2 = add_2_to_duplicates(TodaysData2)

## Creates simple dataset to see the expected stats of all the predicted metrics

In [63]:
# Performs a two "sample" z test to see the likelihood of a quality start occurring
def row_z_score(row):
    x_bar1 = row["xRA"]
    x_bar2 = row["xOuts"]
    mu1 = 3
    mu2 = 18
    sigma1 = Train3["RA"].std()
    sigma2 = Train3["Outs"].std()
    n1 = len(TodaysData2)
    n2 = n1
    
    # Calculate z-scores
    z_score_xRA = (x_bar1 - mu1) / (sigma1 / np.sqrt(n1))
    z_score_xOuts = (x_bar2 - mu2) / (sigma2 / np.sqrt(n2))
    
    # Calculate probabilities, want less than
    prob_xRA_less_than_mu1 = 1 - stats.norm.cdf(z_score_xRA)
    prob_xOuts_greater_than_mu2 = stats.norm.cdf(z_score_xOuts)

    # Combine probabilities by multiplying it
    combined_prob = prob_xRA_less_than_mu1 * prob_xOuts_greater_than_mu2
    return combined_prob

# Calculate the chance of a quality start occurring and finds the xQS points
percent = TodaysData2.apply(row_z_score, axis=1)
xQS = percent * 5

# Adds the xQS to the xFS
TodaysData2.loc[:, "xFS"] = (xQS + TodaysData2["xK"] * 3 + TodaysData2["xOuts"] - TodaysData2["xRA"] * 3)
TodaysData3 = TodaysData2[["BatterTeam", "SP", "Starts", "xK", "xBB", "xHA", "xRA", "xOuts", "xFS"]].round(2)
TodaysData3.sort_values("xFS")

,BatterTeam,SP,Starts,xK,xBB,xHA,xRA,xOuts,xFS
10,HOU,Josh Winckowski,5.0,1.34,1.00,4.17,1.69,10.61,9.57
24,ATL,Dakota Hudson,17.0,2.99,3.00,5.80,3.19,14.88,14.29
19,MIL,Nick Martinez,6.0,3.70,1.00,5.77,3.61,16.18,16.47
22,CHC,Chris Flexen,21.0,3.89,2.00,5.22,3.35,15.11,16.74
8,SD,Roddery Munoz,13.0,4.70,2.57,5.12,3.51,14.89,18.45
12,LAA,Patrick Corbin,23.0,4.14,1.01,6.33,3.20,16.14,18.97
15,KC,Andre Pallante,10.0,3.98,2.00,4.25,2.47,14.47,19.00
13,WSH,Griffin Canning,23.0,4.00,2.00,4.92,2.89,16.01,19.36
4,OAK,Yariel Rodriguez,11.0,4.03,2.59,3.31,1.41,11.82,19.67
2,NaN,Gerrit Cole,8.0,5.02,1.00,5.29,3.24,15.14,20.47


## Using Prizepicks API to load in data to see lines to find edge

In [64]:
def call_endpoint(url, max_level=3, include_new_player_attributes=False):
    '''
    takes: 
        - url (str): the API endpoint to call
        - max_level (int): level of json normalizing to apply
        - include_player_attributes (bool): whether to include player object attributes in the returned dataframe
    returns:
        - df (pd.DataFrame): a dataframe of the call response content
    '''
    resp = requests.get(url).json()
    data = pd.json_normalize(resp['data'], max_level=max_level)
    included = pd.json_normalize(resp['included'], max_level=max_level)
    if include_new_player_attributes:
        inc_cop = included[included['type'] == 'new_player'].copy().dropna(axis=1)
        data = pd.merge(data
                        , inc_cop
                        , how='left'
                        , left_on=['relationships.new_player.data.id'
                                   ,'relationships.new_player.data.type']
                        , right_on=['id', 'type']
                        , suffixes=('', '_new_player'))
    return data

url = 'https://partner-api.prizepicks.com/projections?league_id=2&per_page=1000'
df = call_endpoint(url, include_new_player_attributes=True)

prizepicks1 = df[["attributes.description", "attributes.line_score", "attributes.odds_type", "attributes.stat_type", "attributes.name", "attributes.position", "attributes.team"]]
prizepicks1 = prizepicks1[(prizepicks1["attributes.position"] == "P") & ((prizepicks1["attributes.odds_type"] == "standard") | (prizepicks1["attributes.odds_type"] == "goblin") | (prizepicks1["attributes.odds_type"] == "demon"))].drop(columns = "attributes.position")
prizepicks2 = prizepicks1.rename(columns={'attributes.name': 'SP', 'attributes.description': 'BatterTeam', 'attributes.team': 'PitcherTeam',  "attributes.line_score": "Line", "attributes.stat_type": "Stat", "attributes.odds_type": "Type"})

## Strikeouts

In [65]:
StrikeoutLine = prizepicks2[(prizepicks2["Stat"] == "Pitcher Strikeouts") & (prizepicks2["Type"] == "standard")]
Strikeouts1 = TodaysData3[["BatterTeam", "SP", "Starts", "xK"]]
Strikeouts2 = pd.merge(Strikeouts1, StrikeoutLine[["SP", "Line", "Stat", "Type"]], left_on = ['SP'], right_on = ['SP'], how = 'left').dropna()
Strikeouts2["Edge"] = Strikeouts2["xK"] - Strikeouts2["Line"]
Strikeouts2["Edge%"] = abs(Strikeouts2["Edge"] / Strikeouts2["Line"]) * 100
Strikeouts2.sort_values("Edge%").round(2)

,BatterTeam,SP,Starts,xK,Line,Stat,Type,Edge,Edge%
23,CWS,Justin Steele,18.0,5.99,6.0,Pitcher Strikeouts,standard,-0.01,0.17
30,NYM,Logan Gilbert,23.0,6.70,6.5,Pitcher Strikeouts,standard,0.20,3.08
12,LAA,Patrick Corbin,23.0,4.14,4.0,Pitcher Strikeouts,standard,0.14,3.50
18,CIN,Tobias Myers,16.0,4.69,4.5,Pitcher Strikeouts,standard,0.19,4.22
26,PHI,Zac Gallen,19.0,4.76,5.0,Pitcher Strikeouts,standard,-0.24,4.80
17,MIN,Gavin Williams,7.0,5.84,5.5,Pitcher Strikeouts,standard,0.34,6.18
28,PIT,River Ryan,3.0,4.69,5.0,Pitcher Strikeouts,standard,-0.31,6.20
19,MIL,Nick Martinez,6.0,3.70,4.0,Pitcher Strikeouts,standard,-0.30,7.50
21,TB,Corbin Burnes,23.0,6.00,6.5,Pitcher Strikeouts,standard,-0.50,7.69
14,STL,Michael Wacha,19.0,4.09,4.5,Pitcher Strikeouts,standard,-0.41,9.11


## Walks

In [66]:
WalkLine = prizepicks2[(prizepicks2["Stat"] == "Walks Allowed") & (prizepicks2["Type"] == "standard")]
Walks1 = TodaysData3[["BatterTeam", "SP", "Starts", "xBB"]]
Walks2 = pd.merge(Walks1, WalkLine[["SP", "Line", "Stat", "Type"]], left_on = ['SP'], right_on = ['SP'], how = 'left').dropna()
Walks2["Edge"] = Walks2["xBB"] - Walks2["Line"]
Walks2["Edge%"] = abs(Walks2["Edge"] / Walks2["Line"]) * 100
Walks2.sort_values("Edge%").round(2)

,BatterTeam,SP,Starts,xBB,Line,Stat,Type,Edge,Edge%
21,TB,Corbin Burnes,23.0,1.0,1.5,Walks Allowed,standard,-0.5,33.33
25,COL,Max Fried,19.0,2.0,1.5,Walks Allowed,standard,0.5,33.33
29,LAD,Paul Skenes,14.0,1.0,1.5,Walks Allowed,standard,-0.5,33.33


## Hits Allowed

In [67]:
HitLine = prizepicks2[(prizepicks2["Stat"] == "Hits Allowed") & ((prizepicks2["Type"] == "standard") | (prizepicks2["Type"] == "demon"))]
Hits1 = TodaysData3[["BatterTeam", "SP", "Starts", "xHA"]]
Hits2 = pd.merge(Hits1, HitLine[["SP", "Line", "Stat", "Type"]], left_on = ['SP'], right_on = ['SP'], how = 'left').dropna()
Hits2["Edge"] = Hits2["xHA"] - Hits2["Line"]
Hits2 = Hits2[~((Hits2["Type"] == "demon") | (Hits2["Type"] == "goblin") & (Hits2["Edge"] < 0))]
Hits2["Edge%"] = abs(Hits2["Edge"] / Hits2["Line"]) * 100
Hits2.sort_values("Edge%").round(2)

,BatterTeam,SP,Starts,xHA,Line,Stat,Type,Edge,Edge%
27,AZ,Aaron Nola,23.0,5.60,5.5,Hits Allowed,standard,0.10,1.82
17,MIN,Gavin Williams,7.0,5.58,4.5,Hits Allowed,standard,1.08,24.00


## Runs Allowed

In [68]:
RunLine = prizepicks2[(prizepicks2["Stat"] == "Earned Runs Allowed") & ((prizepicks2["Type"] == "standard") | (prizepicks2["Type"] == "demon"))]
Runs1 = TodaysData3[["BatterTeam", "SP", "Starts", "xRA"]]
Runs2 = pd.merge(Runs1, RunLine[["SP", "Line", "Stat", "Type"]], left_on = ['SP'], right_on = ['SP'], how = 'left').dropna()
Runs2["Edge"] = Runs2["xRA"] - Runs2["Line"]
Runs2 = Runs2[~((Runs2["Type"] == "demon") | (Runs2["Type"] == "goblin") & (Runs2["Edge"] < 0))]
Runs2["Edge%"] = abs(Runs2["Edge"] / Runs2["Line"]) * 100
Runs2.sort_values("Edge%").round(2)

,BatterTeam,SP,Starts,xRA,Line,Stat,Type,Edge,Edge%
27,AZ,Aaron Nola,23.0,2.43,2.5,Earned Runs Allowed,standard,-0.07,2.8
13,WSH,Griffin Canning,23.0,2.89,2.5,Earned Runs Allowed,standard,0.39,15.6
19,MIL,Nick Martinez,6.0,3.61,2.5,Earned Runs Allowed,standard,1.11,44.4


## Pitching Outs

In [69]:
OutLine = prizepicks2[prizepicks2["Stat"] == "Pitching Outs"]
Outs1 = TodaysData3[["BatterTeam", "SP", "Starts", "xOuts"]]
Outs2 = pd.merge(Outs1, OutLine[["SP", "Line", "Stat", "Type"]], left_on = ['SP'], right_on = ['SP'], how = 'left').dropna()
Outs2["Edge"] = Outs2["xOuts"] - Outs2["Line"]
Outs2 = Outs2[~((Outs2["Type"] == "demon") | (Outs2["Type"] == "goblin") & (Outs2["Edge"] < 0))]
Outs2["Edge%"] = abs(Outs2["Edge"] / Outs2["Line"]) * 100
Outs2.sort_values("Edge%").round(2)

,BatterTeam,SP,Starts,xOuts,Line,Stat,Type,Edge,Edge%
29,LAD,Paul Skenes,14.0,17.87,17.5,Pitching Outs,goblin,0.37,2.11
27,AZ,Aaron Nola,23.0,17.97,17.5,Pitching Outs,goblin,0.47,2.69
13,WSH,Griffin Canning,23.0,16.01,16.5,Pitching Outs,standard,-0.49,2.97
22,CHC,Chris Flexen,21.0,15.11,14.5,Pitching Outs,goblin,0.61,4.21
12,LAA,Patrick Corbin,23.0,16.14,17.0,Pitching Outs,standard,-0.86,5.06
18,CIN,Tobias Myers,16.0,15.44,16.5,Pitching Outs,standard,-1.06,6.42
31,SEA,Sean Manaea,22.0,15.68,17.5,Pitching Outs,standard,-1.82,10.40
15,KC,Andre Pallante,10.0,14.47,17.0,Pitching Outs,standard,-2.53,14.88


## Fantasy Score

In [70]:
FSLine = prizepicks2[prizepicks2["Stat"] == "Pitcher Fantasy Score"]
FS1 = TodaysData3[["BatterTeam", "SP", "Starts", "xFS"]]
FS2 = pd.merge(FS1, FSLine[["SP", "Line", "Stat"]], left_on = ['SP'], right_on = ['SP'], how = 'left').dropna()
FS2["Edge"] = FS2["xFS"] - FS2["Line"]
FS2["Edge%"] = abs(FS2["Edge"] / FS2["Line"]) * 100
FS2.sort_values("Edge%").round(2)

,BatterTeam,SP,Starts,xFS,Line,Stat,Edge,Edge%
30,NYM,Logan Gilbert,23.0,35.39,35.5,Pitcher Fantasy Score,-0.11,0.31
29,LAD,Paul Skenes,14.0,39.19,38.5,Pitcher Fantasy Score,0.69,1.79
27,AZ,Aaron Nola,23.0,30.94,30.0,Pitcher Fantasy Score,0.94,3.13
23,CWS,Justin Steele,18.0,33.56,35.0,Pitcher Fantasy Score,-1.44,4.11
21,TB,Corbin Burnes,23.0,33.44,38.0,Pitcher Fantasy Score,-4.56,12.00
25,COL,Max Fried,19.0,29.51,34.0,Pitcher Fantasy Score,-4.49,13.21
14,STL,Michael Wacha,19.0,22.82,27.0,Pitcher Fantasy Score,-4.18,15.48
18,CIN,Tobias Myers,16.0,23.28,28.0,Pitcher Fantasy Score,-4.72,16.86
26,PHI,Zac Gallen,19.0,23.85,31.0,Pitcher Fantasy Score,-7.15,23.06
31,SEA,Sean Manaea,22.0,25.08,33.5,Pitcher Fantasy Score,-8.42,25.13


In [71]:
Props1 = pd.concat([Strikeouts2, Hits2, Runs2, Outs2], ignore_index = True)
Props2 = Props1[(Props1["Edge%"] > 10) & (Props1["Starts"] > 3)]
Props2 = Props2[["BatterTeam", "SP", "Starts", "Stat", "Type", "Line", "Edge", "Edge%"]].sort_values("Edge%").round(2)
Props2

,BatterTeam,SP,Starts,Stat,Type,Line,Edge,Edge%
28,SEA,Sean Manaea,22.0,Pitching Outs,standard,17.5,-1.82,10.40
7,CHC,Chris Flexen,21.0,Pitcher Strikeouts,standard,3.5,0.39,11.14
9,COL,Max Fried,19.0,Pitcher Strikeouts,standard,6.0,-0.68,11.33
2,KC,Andre Pallante,10.0,Pitcher Strikeouts,standard,3.5,0.48,13.71
13,LAD,Paul Skenes,14.0,Pitcher Strikeouts,standard,7.0,1.00,14.29
23,KC,Andre Pallante,10.0,Pitching Outs,standard,17.0,-2.53,14.88
18,WSH,Griffin Canning,23.0,Earned Runs Allowed,standard,2.5,0.39,15.60
15,SEA,Sean Manaea,22.0,Pitcher Strikeouts,standard,6.0,-0.98,16.33
11,AZ,Aaron Nola,23.0,Pitcher Strikeouts,standard,5.0,0.98,19.60
16,MIN,Gavin Williams,7.0,Hits Allowed,standard,4.5,1.08,24.00


In [ ]:
excel_file = 'Pitcher-Tracker.xlsx'

# If deleting the mode and engine it rewrites the whole file
with pd.ExcelWriter(excel_file, mode='a', engine='openpyxl') as writer:
    Props2.reset_index().to_excel(writer, index=False, sheet_name = eastern_time.strftime("%m-%d-%y"))